# Advanced Bokeh Plots

- Gaussian visualization with sliders
- Multiple plots side by side

In this notebook, we are going to look at adding interactivity to plots. One type of interactivity is to add panning to multiple plots. Another type of interactivity is to add buttons and controls to plots. Both of them are possible in bokeh. 

In [ ]:
# Import all the libraries and modules used in this notebook:
from bokeh.plotting import figure 
from bokeh.io import output_notebook, show
output_notebook()
from bokeh.layouts import gridplot, column, layout, row
from bokeh.models import Slider, CustomJS, ColumnDataSource
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np 

Now let's start with the first one - 

#### 1) Linking panning in multiple plots 

Let us go back to first notebook and get our sine wave function. Suppose we want to see the scatter plot and the line plot in the same figure. How can we do that? For that we will use the 'gridplot' command. 

If we share the x-range and the y-range of the plot then the plots will be linked. To test this out, run the code below- 

```python 
import numpy as np 
x = np.linspace(-10, 10, 100)
y = np.cos(x)


# scatter plot 
scatter_plot = figure(width=500, height=500)
scatter_plot.circle(x, y, size=7, color="green")

line_plot = figure(width=500, height=500, x_range=scatter_plot.x_range, y_range=scatter_plot.y_range)
line_plot.line(x, y, color="green")

p = gridplot([[scatter_plot, line_plot]])
show(p)


```




In [ ]:
# run the above code for panning plots:



#### 2) Adding a slider to a plot

Building interactive plots in bokeh is easy since bokeh has its own collection of widgets that you can use. For example, if we want to build a plot that let us control the wavelength and other parameters of the sine wave, we would write the following code- 

```python
x = np.linspace(-20, 20, 100)
y = np.cos(x)

source = ColumnDataSource(data=dict(x=x, y=y))

plot = figure(plot_width=400, plot_height=400, x_range=[-20,20])
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)


slider = Slider(start=0.5, end=4, value=1, step=.1, title="wavelength")

callback = CustomJS(args=dict(source=source), code="""
    var data = source.data;
    var slider_value = cb_obj.value
    var x = data['x']
    var y = data['y']
    for (var i = 0; i < x.length; i++) {
        y[i] = Math.cos(x[i]/slider_value)
    }
    source.change.emit();
""")

slider.js_on_change('value', callback)

layout = column(slider, plot)

show(layout)


```

Notice in this code we have used the command 'ColumnDataSource'. This is bokeh's own data dictionary like object which lets you provide data to a plot and share data between plots. There are various pieces that we need to cover in this specific example. The first part of the code is fairly straight forward. We pass 'x' and 'y' to 'ColumnDataSource' which let us create the data object source. We plot the line plot using 'x' and 'y' as strings and 'source' as the data source in 'plot.line'. The next set of lines can be a bit confusing so we will go over them carefully. 

First we start by declaring a slider object. We define the starting and ending value of the slider, the steps it should take and the title of the slider. Then we define what is known as the callback. A callback is a piece of code the slider will use to operate. We need this because, we need to instruct the program what to change when we slide the slider. 

This part of the code is written in JavaScript. This is a drawback of bokeh that to some extend one has to rely on JavaScript functions to build interactivity and add new functionality. The code however is rather straight forward. 

We have imported the command 'CustomJS' which takes two arguments, it takes a data source which must be bokeh 'ColumnDataSource' object and it takes the code that it needs to run in order to make the slider work. 

This code is- 
```Python 

code="""
    var data = source.data;
    var slider_value = cb_obj.value
    var x = data['x']
    var y = data['y']
    for (var i = 0; i < x.length; i++) {
        y[i] = Math.cos(x[i]/slider_value)
    }
    source.change.emit();
""")

```

Now let's break this down further. The source of the data is our 'ColumnDataSource' which contains both 'x' and 'y' values. Hence the first line of the code is  

```Python
"""var data = source.data;"""

```
In JavaScript whenever we declare a variable, unlike python we must use the declaration 'var'. If it is a var then it can change. If we do not want the value of the object to change we can set it to a 'const'. In all variable declarations we are using var since we expect these values to vary. 

In the second line of code, we set the current value of the slider. 

```Python
"""var slider_value = cb_obj.value"""

```


'cb_obj' is the slider object and 'cb_obj.value' is its value at the current state. 

The next two lines are just us getting the x and the y values from the 'ColumnDataSource' object that was passed to the JavaScript object data. 
```Python
""" var x = data['x']
    var y = data['y']
"""

```

The next line involves running a for loop in JavaScript. Unlike python where we can run a for loop by using indentation in JavaScript you need to keep code on which you will run a for loop in curly braces. Further more unlike a python for loop the way we specify conditions for a for loop are different in JavaScript. 


```Python
""" 
  for (var i = 0; i < x.length; i++) {
        y[i] = Math.cos(x[i]/slider_value)
    }
"""

```

The first part of the for loop is - 

```Python 
 """(var i = 0; i < x.length; i++)"""

```
where we are providing three pieces of information - 

1) **'var i = 0'** says that our for loop starts at i=0. We are declaring this variable i which is valid only in this for loop and nowhere else

2) **'i < x.length'** this condition says that the value of i must not exceed the total number of points in 'x'. This means that suppose we have a plot for 500 points, 'i' must not have a value greater than 499 points since our plot has no information beyond that many points. 

3) **'i++'** is JavaScript's way of writing
```Python 
 i = i+1 
```
   
So what is there inside the for loop. We have code that calculates the new y values given the x values and the slider values as well. For a cosine or sine wave, when we divide the argument by some number it changes the wavelength of the wave. In our case, that number is provided by the value as we change the slider.

Once this calculation has been done, last line of the code 'source.change.emit();' is used to pass the changes back to python.

Coming back to the python code. The line - 

```Python
slider.js_on_change('value', callback)

```
is used to call the callback code so that we can change the values on the plot. This is why we have the string term 'value' in the function '.js_on_change()'. 

We then add the plot and the slider object in a bokeh layout object called 'column'. This plots the objects in a column form. You can also plot them as a row object.


Here is a question to see if you can play around with the code 

**Question** 

Multiplying the cosine value with a number changes the amplitude of a cosine wave. Building upon the code we have for wavelength, create a slider for amplitude and display a plot where you can change both the amplitude and wavelength of a sine wave. 




In [ ]:
# solution 
x = np.linspace(-20, 20, 100)
y = np.cos(x)

source = ColumnDataSource(data=dict(x=x, y=y))

plot = figure(plot_width=400, plot_height=400, y_range=[-5,5])
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)


wavelength_slider = Slider(start=0.5, end=4, value=1, step=.1, title="wavelength")
amplitude_slider = Slider(start=0.5, end=4, value=1, step=.1, title="amplitude")

wavelength_callback = CustomJS(args=dict(source=source, amplitude=amplitude_slider, wavelength=wavelength_slider),
    code="""
    var data = source.data;
    const amp = amplitude.value
    const wav = wavelength.value 
    var x = data['x']
    var y = data['y']
    for (var i = 0; i < x.length; i++) {
        y[i] = amp*Math.cos(x[i]/wav)
    }
    source.change.emit();
""")

wavelength_slider.js_on_change('value', wavelength_callback)
amplitude_slider.js_on_change('value', wavelength_callback)


p = column(wavelength_slider,amplitude_slider, plot)

show(p)


All we did in the solution to the above practice problem was that we added a new slider to the same callback. Also notice the fact that we fixed the 'y_range' of the plot, we did this since we cannot see the change in amplitude if we do not add this. Bokeh will automatically rescale the plot to include a new range of values, hence we fix the value of x range. 

There is a very nice bokeh library demo for the sine wave here - 

https://docs.bokeh.org/en/latest/docs/gallery/slider.html

Take a look at it for adding multiple sliders. It deals with the same example but with more variables. 


If you feel uncomfortable going down the JavaScript path build and use widgets, we can also use Ipython widgets with bokeh plots. Let us look the slider example but with Ipython widgets. 


```Python

def get_wavelength(wavelength): 

    x = np.linspace(-20, 20, 100)
    y = np.cos(x/wavelength)
    
    plot = figure(plot_width=400, plot_height=400, x_range=[-20,20])
    plot.line(x, y, line_width=3, line_alpha=0.6)

    show(plot)

    return None 


slider_obj =  widgets.FloatSlider(value=0.5, min=0.1, max=5,step=0.0001)
interact(get_wavelength, wavelength=slider_obj);
    
```

The code here is a bit easier than the JavaScript part. We start by writing a function called 'get_wavelength()'. This is a callback similar to what we have earlier. Rather than a JavaScript callback, we have a python callback. In this callback function we are passing the value of the slide using the variable 'wavelength'. 

Inside the function we plot the sine wave as we would normally do with a bokeh plot with the addition that we divide the x value by the wavelength so that whenever the value of wavelength changes, a new plot is generated. The function would return nothing since the plot is being displayed within the function itself. 

Things get interesting with the interact statement. What interact does is that whenever we change the value of the slider, it runs the function 'get_wavelength()' and updates the plot. 'interact' takes in the 'get_wavelength()' function, it also takes in value of 'wavelength' which is provided by the slider object.  

The slider object takes in multiple arguments similar to the bokeh slider object. It takes in the starting, ending values and the step size of the slide values. 

Run the above code in the cell below to see how the plot looks


In [ ]:
# run ipython widget code below:


What you will notice after running the code is that as you change the slider values, there will be somewhat of a flicker. This is because Ipython widgets is updating the plot each time as you change the value of the wavelength. One way to get around this problem is to use 'interact_manual' what this does is to plot the sine wave only when you press a button. Here is an example- 



```Python

def get_wavelength(wavelength): 

    x = np.linspace(-20, 20, 100)
    y = np.cos(x/wavelength)
    
    plot = figure(plot_width=400, plot_height=400, x_range=[-20,20])
    plot.line(x, y, line_width=3, line_alpha=0.6)

    show(plot)

    return None 


slider_obj =  widgets.FloatSlider(value=0.5, min=0.1, max=5,step=0.0001)
interact_manual.opts["manual_name"]="Plot"
interact_manual(get_wavelength, wavelength=slider_obj);
    
```

Run the code in the cell below to see the result

Note: We have changed the name of the ipython widget button to plot reflect to the function of the button


In [ ]:
# run the code for interact_manual below:


Now you will notice that you can change the value of the slider then click plot to see the plot with a new value of the wavelength. A third way of doing this is by turning continuous update off in Ipython widgets. For that, we can do the following- 

```Python 
def get_wavelength(wavelength): 

    x = np.linspace(-20, 20, 100)
    y = np.cos(x/wavelength)

    plot = figure(plot_width=400, plot_height=400, x_range=[-20,20])
    plot.line(x, y, line_width=3, line_alpha=0.6)

    show(plot)

    return None 


slider_obj =  widgets.FloatSlider(value=0.5, min=0.1, max=5,step=0.0001,continuous_update=False)
interact.opts["manual_name"]="Plot"
interact(get_wavelength, wavelength=slider_obj);
```

In this case we have to add 'continuous_update=False' condition to the slider itself. Run the code above in the cell below to see the result



In [ ]:
# run code for slider with continuous update set to false


Adding other widgets and interactive elements is fairly similar to that of the slider. For this purpose checkout the documentation for bokeh widgets at - 
https://docs.bokeh.org/en/latest/docs/user_guide/interaction/widgets.html#userguide-interaction-widgets





#### Bokeh widgets vs Ipython widgets

So far we have shown you examples of changing plots with a Bokeh slider and an Ipython slider. There are some things to be aware as you choose which one you want to go with. 

1) Bokeh widgets are great since they work directly with bokeh plots but as mentioned you are going to have to learn a bit of javascript to build more complicated visualization. This is a bit of a double edged sword since, since callbacks and extra code is optimized for bokeh it is definitely faster than using Ipython widgets. JavaScript can introduce an added level of difficulties and debugging may not be that easy. 

2) On the other hand Ipthyon widgets provide a python and jupyter friendly way of building interactive plots. However as we mentioned since the widgets are external to bokeh, they will have to replot a figure every time. This can slow down your visualization considerably, especially when you have large dataset, even dataset with hundreds of MBs will struggle with this approach. Another place this will fail is when you are plotting streaming data, then plotting data will not work well with Ipython widgets. 


### When to use what? 

Over the course of this section and course, you have come across multiple ways that you can plot data. You have seen examples of - 

1) Matplotlib <br>
2) Plotly <br>
3) Bokeh <br> 
4) Other sources <br> 

It can become confusing as to which package to use when. This is where some of the guidelines below come in handy. 

- <b>Matplotlib</b> <br><br> Use matplotlib for quick and dirty results. There are going to be times when you need results on the fly, you do not need  advanced interactivity but a simple visualization. Combining matplotlib with %matplotlib notebook after import statements allows for simple interactive matplotlib figure (but only in jupyter notebook). So make sure that you have that in place for creating easy interactive plots in jupyter notebook. You can also use Ipython widgets with matplotlib plots which is useful. Matplotlib does have a lot of options which are useful. You can easily export matplotlib files into various formats and create animation with matplotlib as well. Not to mention that since matplotlib is older, there is good documentation on its features and there is a lot of help available online for beginners. <br><br>  There are some drawbacks to matplotlib. The quality of your exported plot may not be too good. The interactivity of matplotlib is limited. For presentations and talks matplotlib plots just don't look good. There is some 3d plot support as well in matplotlib but it has its limitation. Another drawback is that to add plotly and bokeh type features it requires a lot of code.


- <b>Plotly</b> <br><br> Plotly is by far the easiest library to work with for visualizations. Each Plotly plot comes with basic elements of interactivity- the ability to hover over data points, the ability to change hover tooltip values, an easy way to implement interactive legends etc. This means the amount of code we have to write in order to have an interactive and useful plotly plot is minimal. It also has many different plotly types available. It has its own library for widgets similar to bokeh and it covers a broad range of use cases. <br><br> Plotly has two ways you can plot plots quickly. One way is to use [plotly express](https://plot.ly/python/plotly-express/). Plotly express is fairly new and allows you to design plots in a single line of code.It comes with a lot of guard rails so you can focus on working with your data rather than configuring your plot. There is also a third party library for plotly called [Cufflinks](https://plot.ly/python/v3/ipython-notebooks/cufflinks/) which acts like plolty express. <br><br> Another big advantage of plotly is plotly-dash. Plotly dash is a framework to create visualization applications that can be deployed on a website, you can use plotly  widgets with plotly dash. Bokeh as a similar system called bokeh server. Plotly Dash is primarily used to build deployable dashboards for monitoring processes but it can be used to host almost any type of plotly interactive visualizations. <br><br> While plotly is extremely powerful, it has its drawbacks. Plotly can be slow with large datasets. There are certain aspects of plot customization that are not easy to implement in plotly. It is generally advisable to use plotly express or cufflinks rather than vanilla plotly for quick plots but learning vanilla plotly is crucial especially when you want to build plotly dash applications. The sytax for plotly plots frustratingly does not always carry over to plotly dash. The documentation can be unclear with certain types of plots. Most of plotly features we have described are based on plotly offline. Plotly also has an online version where you can store charts and even get organization support for Plotly dash, however this is extremely expensive. 


- <b>Bokeh</b> <br><br> As discussed in the beginning of this section, an alternate to plotly is bokeh. Both plotly and bokeh share many similar features however there is more customization in bokeh compared to plolty. One really nice feature in bokeh is that you can specify which interactivity tools you want on the plot. Bokeh also has certain features like patches that plotly does not have. For example, with patches you can draw any type of shapes easily. We use this feature especially in the math and stat section where we talk about the area under the curve. There are other annotation features and plotting features that bokeh has that plotly does not carry. A major advantage that bokeh has over plotly is that it can handle large datasets. It also integrates well with holoviews which is also good at handling large datasets.<br><br> Similar to plotly dash, bokeh can also be used to build and serve applications. It has similar features to plotly dash and can be used to build dashboards and stand alone interactive visualizations. Bokeh's dashboarding framework and bokeh in general is very well documented hence if you are willing to go through the documentation you can pretty much learn bokeh fairly quickly. <br><br> One of the main drawbacks of bokeh is that unlike plotly, it does not support anything like plotly express. You can use a third party tool like holoviews which support bokeh extensively but within bokeh there is nothing that will help you plot bokeh plots quickly. There are certain types of plots where this can be a huge problem. Take for example, the heatmap plot. Plotly has a clean way to provide data to a function that will plot a heatmap and you can customize parts of the plot. Plotting heatmaps in bokeh can be extremely frustration since you need to construct the plot element by element. This allows for more control on the plots but it can also be challenging when you are trying to get a plot up and running quickly. Another issue is that to add callbacks and more interactivities to plots you will have to use JavaScript. While there are good documented examples of how to use JavaScript in bokeh callbacks, it can be a bit jarring to go to JavaScript which is working on a python application. 



- <b>Other alternatives</b> <br><br> Apart from the three big visualization packages that we have covered so far there are others that are available for python. We have already seen some examples of [seaborn](https://seaborn.pydata.org/). Seaborn is heavily used with matplotlib and pandas. It comes in handy in generating good looking plots. [Holoviews](http://holoviews.org/) is another package that specializes simple plotting syntax and easy to understand code for plotting. Another plotting library that is being currently developed for jupyter notebooks is [bqplot](https://github.com/bloomberg/bqplot). 




Ultimately, there are many libraries and tools that you can use to visualize data. The important point is to keep in mind the purpose of the visualization. Adding interactivity or buttons to a plot when it's not needed distracts from the core purpose of the visualization which is to communicate data in a visual manner for easy inference. As you design multiple data visualizations keep in mind that the visualizations should be able to tell the story of the data. Check out our micro-course on data stories for an example of it! 

